In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
datapath = "./data/"

In [2]:
df = pd.read_csv(datapath + "Fuel_Consumption_2000-2022.csv") # https://www.kaggle.com/datasets/ahmettyilmazz/fuel-consumption
df_prices = pd.read_csv(datapath + "Fuel_prices_Canada.csv") # https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=1810000101

In [ ]:
df_prices.head()